# Web Scraping de Webs Estáticas

Leonardo Gonzalez 

Contacto:leo.gonzalez11@gmail.com


"Cargando" la pagina


In [ ]:
#importamos las librerias que vamos a utilizar para leer la pagina web
import requests  # metodo que para leer html evitando errores de lectura, antes recuerda intalar
import pprint    #esto nos premite imprimir cosas mas bonitas nada mas.

In [ ]:
#Traemos la pagina y la guardamos con el metodo get
url = 'https://www.buscalibre.cl/libros/search?q=python'
page= requests.get(url)
#el metodo get su utiliza para leer datos existe otro metodos como el post y put

In [ ]:
#verificamos que tipo de respuesta tenemos 
page

Tenemos dos manera de leer una pagiana mediante el 

In [ ]:
#page.text

In [ ]:
#pprint.pprint(page.content)

In [ ]:
#importamos la libreria beautifulsoup4
from bs4 import BeautifulSoup

In [ ]:
#Ahora utilizaremos la libreria para extraer los datos html de la pagina que queremos.
#con soup pordemos recorrer el html
soup = BeautifulSoup(page.text, "html") #combierte el html en texto

In [ ]:
soup.title

In [ ]:
#Es posible ademas repasar el contenido de la pagina para mejorar la lectura y acceder a los niveles
soup = BeautifulSoup(page.content, "html.parser")
#argumento parser ayuda a recorrer el html
#https://code.tutsplus.com/es/tutorials/scraping-webpages-in-python-with-beautiful-soup-the-basics--cms-28211
# soup=BeautifulSoup(page.content) tambien sirve

In [ ]:
#Exploramos los resultados
results = soup.find_all('div',class_='producto')
# table=soup.find_all('table')
#buscamos todos los elementos que esten marcados con las clase producto 
print('articulos por pagina'+" "+str(len(results)))

In [ ]:
#podemos visualizar de manera comoda lo que estamos haciendo con pprint

#traemos los datos del elemento 0 para reviasr que este correcto
print(results[0])

# print(soup.prettify()) si queremos ver todos los datos

In [ ]:
#Ahora queremos obtener una etiqueta dentro de un elemento de soup
element = results[0]
#creamos una var del titulo del libro
book_title = element.find('a')['title'] #find devuelve el primer valor de un elemento
print(book_title)

In [ ]:
#Queremos obtener el precio 
element.find('div',class_='box-precio-v2').text #div solo pueden tener un atributo class

In [ ]:
element.find('div',class_='autor').text

In [ ]:
#atributos custom 
print(element['class'])

In [ ]:
print(element['data-precio'])

In [ ]:
#Obteniendo data de un elemento anidado
#todo los elemento se obtiene con find_all
print(element.find_all('div', class_='descuento-v2')[0].text)

In [ ]:
#probamos un mensaje de error por que quremos recorrer todas las paginas posible hasta la ultima

#buscamos un url que nos de erro
URL = 'https://www.buscalibre.cl/libros/search?q=bailarina&page=100' #pagina 100
response = requests.get(URL)
not_found = 'Lo sentimos, pero no encontramos lo que buscas'
print(response.status_code)
print (not_found not in response.text)

# Tenemos todo lo necesario por unir todo y crear nuestro df

In [ ]:
#creamos las variables que queremos segun nuestro repaso anterior
titulo=[]
autor=[]
precio=[]
descuento=[]
#tenemos el mensaje de error para cada pagina a recorer
not_found_msg = 'Lo sentimos, pero no encontramos lo que buscas'
URL = 'https://www.buscalibre.cl/libros/search?q=python&page'
pag_index = 1 #definimos en que pagina comenzamos la busqueda

#utilizamos un bucle while para cumplir la condicion para 
while (True):
    URL = 'https://www.buscalibre.cl/libros/search?q=python&page={0}'.format(pag_index) #recorrer las paginas
    response = requests.get(URL)
    #si el mensaje que definimos no aprece entonces
    if (not_found_msg not in response.text):
        #repasa el html
        soup = BeautifulSoup(response.content, 'html.parser')
        
        #busca todos los elementos que esten marcados con las clase producto 
        results = soup.find_all('div',class_='producto')
        
        #con el metodo for y append iteramos y rellenamos las listas
        for element in results:
            titulo.append(element.find('a')['title'])
            autor.append(element.find('div',class_='autor').text)
            precio.append(element['data-precio'])
            descuento.append(element.find_all('div', class_='descuento-v2')[0].text)
            pag_index+= 1
    else:
        break;

In [ ]:
print(titulo)

In [ ]:
print(autor)

In [ ]:
#para terminar importamos pandas para crear un df
import pandas as pd

#creamos el data con los encebezados que queramos
data = {'titulo' : titulo, 'autor':autor, 'precio':precio, 'descuento':descuento}
#creamos el df
df = pd.DataFrame(data)
df.head()

In [ ]:
# remover el % de la columan dscto
df['descuento'] = df['descuento'].apply(lambda x : x[:x.find('%dcto'):])
df.head()